In [3]:
import glob
import pandas as pd

In [4]:
csv_files = glob.glob( '*/**.csv' )

In [22]:
activations = pd.read_csv(csv_files[0], sep=';', 
                          dtype={'SUBS_ID': str, 'PROD_ID': int}, 
                          dayfirst=True,
                          parse_dates=['START_DTTM', 'END_DTTM'])
sales = pd.DataFrame()
for csv in csv_files[1:]:
    df = pd.read_csv(csv, sep = ';')
    df = (
        df
         .query("SUBS_ID.isnull() == False")
         .assign(SUBS_ID = 'id' + df.SUBS_ID.str.removeprefix("id"))
    )
    sales = pd.concat((sales, df), sort=True, ignore_index=True)

merged = activations.merge(sales, how = 'inner', on = ['SUBS_ID', 'PROD_ID'])

In [23]:
(
    merged
     .assign(difference = merged.END_DTTM - merged.START_DTTM)
     .query("difference.astype('timedelta64[m]') >= 5")
     .sort_values(by='difference', ascending=False)
     .SUBS_ID.to_list()
)

['id2185490', 'id4651830', 'id5416547', 'id7642700']